## Web Crawling, rottentomatoes 
- Selenium: crawling tool, BeautifulSoup: parsing 
- 대상: 
    - 전체 페이지 DVD&STREAMING MOIVEW(Browse All)  
    https://www.rottentomatoes.com/browse/in-theaters?minTomato=0&maxTomato=100&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=popularity
    - 1) 영화 제목
    - 2) 상영 날짜 
    - 3) 상세 페이지 링크 -> 다음 이동 경로
    
    - 영화 상세 페이지 
    - 1) 평점 및 평가 횟수(TOMATOMETER, AUDIENCE SCORE)
    - 2) MOIVE INFO
        - 1) 개요
        - 2) 장르
        - 3) 평점 => (전문가 평점 횟수/ 전체 평점 횟수) X 전문가 평점 = 비율이 고려된 가중 평균비율(전문가와 일반 모두에게 점수 높은 영화)
        - 4) 감독 
    - 3) CAST
        - 1) 배우 이름
    - 추가가능항목: 평가글, 비평가 등급별 리뷰 횟쉬
- 링크: https://movie.naver.com/movie/point/af/list.nhn


In [14]:
#  packagae download
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

### CH1. 전체 페이지
1) 영화 제목
2) 상세 페이지 링크 -> 해당 링크로 진입하여 크롤링      
3) 상영 가능일 

In [15]:
# Chorme driverload
driver_path =  'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 
url = 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=100&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release' 

browser = webdriver.Chrome(executable_path = driver_path)
browser.get(url)

* 주의) 
    1. page_source를 먼저 불어온 뒤, 작업해야함
    2. browser.quit()할 경우, 데이터를 불러올 수 없음 (항상 사이트가 켜져있는 상태에서 진행) 

In [16]:
time.sleep(3) # page loading 대기 

title = [] 
release_date = []
detail_page = []
browser.page_source

for i in range(35):
    browser.execute_script("return document.body.scrollHeight")
    time.sleep(3)
    browser.find_element_by_css_selector(".btn.btn-secondary-rt.mb-load-btn").click()
    



page_url_base = browser.find_elements_by_css_selector('div.movie_info')

for i in page_url_base:
    title.append(i.find_element_by_css_selector('div.movie_info  .movieTitle').text)
    release_date.append(i.find_element_by_css_selector('div.movie_info  .release-date').text)
    detail_page.append(i.find_element_by_css_selector('div.movie_info a').get_attribute('href'))

browser.quit()

In [17]:
print(f'제목 개수: {len(title)},발행 날짜 개수:{len(release_date)}, 상세페이지 링크 개수: {len(detail_page)}')
detail_page

제목 개수: 32,발행 날짜 개수:32, 상세페이지 링크 개수: 32


['https://www.rottentomatoes.com/m/game_of_death_2020',
 'https://www.rottentomatoes.com/m/retaliation_2020',
 'https://www.rottentomatoes.com/m/fishermans_friends',
 'https://www.rottentomatoes.com/m/the_sunlit_night',
 'https://www.rottentomatoes.com/m/skyman',
 'https://www.rottentomatoes.com/m/babysplitters',
 'https://www.rottentomatoes.com/m/yes_god_yes',
 'https://www.rottentomatoes.com/m/samurai_marathon',
 'https://www.rottentomatoes.com/m/guest_of_honour',
 'https://www.rottentomatoes.com/m/black_magic_for_white_boys',
 'https://www.rottentomatoes.com/m/the_painted_bird',
 'https://www.rottentomatoes.com/m/the_outpost_2020',
 'https://www.rottentomatoes.com/m/dont_be_nice',
 'https://www.rottentomatoes.com/m/dirt_music_2019',
 'https://www.rottentomatoes.com/m/first_cow',
 'https://www.rottentomatoes.com/m/the_prince_2020',
 'https://www.rottentomatoes.com/m/suzi_q',
 'https://www.rottentomatoes.com/m/inmate_1_the_rise_of_danny_trejo',
 'https://www.rottentomatoes.com/m/bless

### 상세페이지 스크랩
    - 1) 청중 평가, 리뷰 횟수 
    - 2) 로튼 토마토 평가, 리뷰 횟수
    - => 청중+토마토 sum(평가)/ 리뷰 회숫 = 가중 평균
    - 3) 평가 단체
    - 4) 장르
    - 5) 감독 

In [ ]:
# Chorme driverload
#driver_path =  'C:/Users/ghkdn/Jupyter/chromedriver_win32/chromedriver' 

time.sleep(5)

Tomatometer_per = []
Tomatometer_cnt = []
Audience_Score_per = []
Audience_Score_cnt = []
Rating = []
Genre = []
Directed_By = []
Release_Date = []

for url in detail_page:
    browser = webdriver.Chrome(executable_path = driver_path)
    browser.get(url)
    browser.page_source

    page_url_base = browser.find_elements_by_css_selector('.mop-ratings-wrap__info')


    for i in page_url_base:
        Tomatometer_per.append(i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(1)  .mop-ratings-wrap__percentage').text)
        Tomatometer_cnt.append(i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(1) .mop-ratings-wrap__review-totals:nth-of-type(1) small.mop-ratings-wrap__text--small').text)

        try:
            Audience_Score_per.append(i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(2) .mop-ratings-wrap__percentage').text)
        except:
            Audience_Score_per.append(None)
        try:
            temp =i.find_element_by_css_selector('.mop-ratings-wrap__info div.mop-ratings-wrap__half:nth-of-type(2) .mop-ratings-wrap__review-totals:nth-of-type(1) strong.mop-ratings-wrap__text--small').text
            Audience_Score_cnt.append(temp[temp.find(":")+2:])
        except:
            print(f"No element!")
            Audience_Score_cnt.append(None)
    print(Tomatometer_per)
    print(Audience_Score_cnt)
    #browser.quit()


    temp ={}


    try: 
        # Rating
        rating_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(1) div.meta-label.subtle')
        rating_label= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(1) div.meta-value')
        if rating_debug.text[:-1] == "Rating":
            Rating.append(rating_label.text)
        else:
            Rating.append(None) 


        # Genre
        genre_label_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(2) .meta-label.subtle')
        genre_label= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(2) .meta-value')
        if genre_label_debug.text[:-1] == "Genre":
            Genre.append(genre_label.text)
        else:
            Genre.append(None) 

        # Directed By
        directed_by_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(3) .meta-label.subtle')
        directed_by_label= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(3) .meta-value')
        if directed_by_debug.text[:-1] == "Directed By":
            Directed_By.append(directed_by_label.text)
        else:
            Directed_By.append(None) 

        # Releas_Date
        release_date_debug = browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(6) .meta-label.subtle')
        release_date= browser.find_element_by_css_selector('.content-meta.info .meta-row.clearfix:nth-of-type(6) .meta-value')
        print(release_date_debug.text)
        print(release_date.text)
        if release_date_debug.text[:-1] == "On Disc/Streaming":
            Release_Date.append(release_date.text)
        else:
            Release_Date.append(None) 

        print(Rating)
        print(Genre)
        print(Directed_By)
        print(Release_Date)
        browser.quit()
    
    except:
        print("No Such Element!")
        continue

In [ ]:
data = pd.DataFrame(title, columns = ["title"]) # ,"release_date"])#,Directed_By)#, columns = ["Rating","Directed_By"])
#data['Release_date'] = Release_Date
#data['detail_page'] = detail_page[:5]
data['Genre'] = Genre
data['Tometer_per'] = Tomatometer_per
data['Tomatometer_cnt'] = Tomatometer_cnt
data['Audience_Score_per'] = Audience_Score_per
data['Audience_Score_cnt'] = Audience_Score_cnt
data['Rating_Compony'] = Rating
data['Directed_By'] = Directed_By

In [157]:
data

,title,Tometer_per,Tomatometer_cnt,Audience_Score_per,Audience_Score_cnt,Rating_Compony,Directed_By
0,Game Of Death,63%,24,28%,50,NR,"Sébastien Landry, Laurence Morais-Lagace"
1,Retaliation (Romans),82%,11,None,None,"R (for disturbing violent/sexual content, lang...","Ludwig Shammasian, Paul Shammasian"
2,Fisherman'S Friends,64%,45,72%,109,"PG-13 (for some strong language, and suggestiv...",Chris Foggin
3,Skyman,67%,15,None,None,NR,Daniel Myrick
4,Zombie For Sale,88%,24,92%,16,NR,Lee Min-jae


In [23]:
# detail_page + "/reviews" = review page
time.sleep(3)

Title = []
Review = [] 
Reviewer = []
Review_date = []

for page in detail_page:
    temp_url = page+"/reviews"
    browser = webdriver.Chrome(executable_path = driver_path)
    browser.get(temp_url)
    

    try: 
        ###### 예외 처리 통과 구문
        browser.find_element_by_css_selector('div.center   .unstyled.articleLink')
        browser.find_elements_by_css_selector('div.the_review')
        browser.find_elements_by_css_selector('div.row.review_table_row a.unstyled.bold.articleLink')
        browser.find_elements_by_css_selector('div.review_area .review-date.subtle.small')
        ##########################
        
        name = browser.find_element_by_css_selector('div.center   .unstyled.articleLink').text


        reivew_boxes = browser.find_elements_by_css_selector('div.the_review')
        for review in reivew_boxes:
            Review.append(review.text)
            Title.append(name) # name 리뷰 개수만큼 넣어주기 

        reviewer_boxes = browser.find_elements_by_css_selector('div.row.review_table_row a.unstyled.bold.articleLink')
        for reviewer in reviewer_boxes:
            Reviewer.append(reviewer.text)

        review_date_boxes = browser.find_elements_by_css_selector('div.review_area .review-date.subtle.small')
        for reviewe_date in review_date_boxes:
            Review_date.append(reviewe_date.text)
        browser.quit()
    except:
        continue

SessionNotCreatedException: Message: session not created
from chrome not reachable
  (Session info: chrome=84.0.4147.89)


In [21]:
review_data = pd.DataFrame(Title,columns= ["Title"])
review_data['Review_date'] = Review_date
review_data["Reviewer"] = Reviewer
review_data["Review"] = Review
review_data

,Title,Review_date,Reviewer,Review
0,THE PRINCE (EL PRINCIPE),"July 24, 2020",James Wegg,Life on the inside
1,THE PRINCE (EL PRINCIPE),"July 21, 2020",Rich Cline,While it's perhaps too full-on for mainstream ...
2,THE PRINCE (EL PRINCIPE),"July 9, 2020",André Hereford,"Gritty, '70s-set gay prison drama El Principe ..."
3,THE PRINCE (EL PRINCIPE),"July 9, 2020",Amy Nicholson,If you would like a movie with really beautifu...
4,THE PRINCE (EL PRINCIPE),"July 4, 2020",Alex Saveliev,The Prince will stay with you long after its c...
...,...,...,...,...
72,RELIC,"July 20, 2020",Matthew Pejkovic,Blends creepy supernatural thrills with nightm...
73,RELIC,"July 20, 2020",Peter Canavese,James summons a dark beauty for this landscape...
74,RELIC,"July 18, 2020",Caillou Pettis,Relic's mysterious and eerie story accompanied...
75,RELIC,"July 17, 2020",Charles Koplinski,The movie is likely to tax even the most patie...


In [161]:
temp_url
browser = webdriver.Chrome(executable_path = driver_path)
browser.get(temp_url)

'https://www.rottentomatoes.com/m/game_of_death_2020/reviews'

#### 기타: 리뷰 페이지 URL 이동(작업중)

In [170]:
temp_url
browser = webdriver.Chrome(executable_path = driver_path)
browser.get(temp_url)

num = 2
while True :
    try:
        print(temp)
        temp = temp_url + "?type=&sort=&page=" + str(num)
        browser.execute_script("return document.body.scrollHeight")
        time.sleep(3)
        browser.find_element_by_css_selector("a.btn btn-xs.btn-primary-rt").click()
        num +=1
        print(temp)
    except:
        print("end page")
        break
    

https://www.rottentomatoes.com/m/game_of_death_2020/reviews?type=&sort=&page=2
end page


In [29]:
import tensorflow as tf
